In [2]:
import pandas as pd
from datetime import datetime
import os

DATA_FILE = "data/students.parquet"

def load_data():
    if os.path.exists(DATA_FILE):
        return pd.read_parquet(DATA_FILE)
    else:
        return pd.DataFrame(columns=["ФИО", "Группа", "Математика", "Физика", "Информатика", "Дата ввода"])

def save_data(df):
    os.makedirs("data", exist_ok=True)
    df.to_parquet(DATA_FILE, index=False)

def add_student(fio, group, math, physics, informatics):
    df = load_data()
    new_row = {
        "ФИО": fio,
        "Группа": group,
        "Математика": int(math),
        "Физика": int(physics),
        "Информатика": int(informatics),
        "Дата ввода": datetime.now()
    }
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.sort_values("ФИО", inplace=True)
    save_data(df)
    print(" Студент добавлен.")

def show_bad_students():
    df = load_data()
    mask = (df[["Математика", "Физика", "Информатика"]] == 2).any(axis=1)
    bad_students = df[mask][["ФИО", "Группа"]]
    if bad_students.empty:
        print(" Студентов с оценкой 2 нет.")
    else:
        print(" Студенты с хотя бы одной двойкой:")
        print(bad_students.to_string(index=False))

def delete_column(column_name):
    df = load_data()
    if column_name in df.columns:
        df.drop(columns=[column_name], inplace=True)
        save_data(df)
        print(f" Колонка '{column_name}' удалена.")
    else:
        print(" Колонка не найдена.")

def delete_student(fio: str):
    global df
    initial_len = len(df)
    
    df = df[df["ФИО"] != fio]
    
    if len(df) < initial_len:
        print(f" Студент '{fio}' удалён.")
        save_data(df)
    else:
        print(f" Студент '{fio}' не найден.")

In [4]:
# Пример использования
add_student("Иванов И.И.", "ИКТ-101", 3, 2, 5)
show_bad_students()
delete_column("Дата ввода")

 Студент добавлен.
 Студенты с хотя бы одной двойкой:
        ФИО  Группа
Иванов И.И. ИКТ-101
Иванов И.И. ИКТ-101
 Колонка 'Дата ввода' удалена.


In [5]:
df = load_data()
delete_student("Иванов И.И.")

 Студент 'Иванов И.И.' удалён.


In [6]:
!pytest test_main.py

============================= test session starts ==============================
platform linux -- Python 3.12.7, pytest-7.4.4, pluggy-1.0.0
rootdir: /home/lyash/workspace/test1
plugins: anyio-4.2.0
collected 1 item                                                               

test_main.py .                                                           [100%]

=============================== warnings summary ===============================
test_main.py::test_add_student
  /home/lyash/workspace/test1/app.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 1 passed, 1 warning in 0